## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

### SECTION 1

### Data Import and Table Creation

In [0]:
# File location and type
file_location = "/FileStore/tables/ipl_ball_by_ball.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "ipl_ball_by_ball"

df.createOrReplaceTempView(temp_table_name)

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "ipl_ball_by_ball"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
# File location and type
file_location = "/FileStore/tables/ipl_matches.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "ipl_matches"

df.createOrReplaceTempView(temp_table_name)

In [0]:
permanent_table_name = "ipl_matches"

In [0]:
# File location and type
file_location = "/FileStore/tables/ipl_venue.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "ipl_venue"

df.createOrReplaceTempView(temp_table_name)

In [0]:
permanent_table_name = "ipl_venue"

### SECTION 2

#### 1) Find the top 3 venues which hosted the most number of eliminator matches?

In [0]:
%sql
select ipl_venue.venue, count(ipl_matches.eliminator) as no_of_eliminator from ipl_venue join ipl_matches on ipl_venue.venue_id=ipl_matches.venue_id where eliminator='Y' group by venue,eliminator  order by no_of_eliminator desc limit 3

#### 2) Return the most number of catches taken by a player in IPL history?

In [0]:
%sql
select count(*) as most_catches from ipl_ball_by_ball where dismissal_kind='caught' group by fielder order by count(*) desc limit 1

#### 3) Write a query to return a report for highest wicket taker in matches which were affected by Duckworth-Lewis’s method (D/L method).

In [0]:
%sql
select bowler, count(is_wicket) from ipl_ball_by_ball join ipl_matches on ipl_ball_by_ball.match_id=ipl_matches.match_id where method='D/L' group by bowler order by count(is_wicket) desc limit 1

#### 4) Write a query to return a report for highest strike rate by a batsman in non powerplay overs(7-20 overs)

In [0]:
%sql
select concat(batsman,' ','-',' ', strike_rate) as Report_Strike_Rate from (select batsman, 100 * ( sum(total_runs) / sum(ball) ) as strike_rate from `ipl_ball_by_ball` where `overs` between 7 and 20 and extras_type not in ('noballs', 'wides') group by batsman order by strike_rate desc limit 1)

#### 5) Write a query to return a report for highest extra runs in a venue (stadium, city).

In [0]:
%sql
select ipl_venue.venue, ipl_venue.city, sum(ipl_ball_by_ball.extra_runs) as extraRuns from ipl_ball_by_ball join ipl_matches on ipl_ball_by_ball.match_id=ipl_matches.match_id join ipl_venue on ipl_matches.venue_id=ipl_venue.venue_id group by venue,city order by extraRuns desc

#### 6) Write a query to return a report for the cricketers with the most number of players of the match award in neutral venues.

In [0]:
%sql
select player_of_match as Player, count(player_of_match) as No_Of_Awards from ipl_matches where neutral_venue = 1 group by Player, neutral_venue order by No_Of_Awards desc

#### 7) Write a query to get a list of top 10 players with the highest batting average

In [0]:
%sql
with 
runs as 
(select batsman as player, sum(total_runs) as total_runs from ipl_ball_by_ball group by batsman),
dismissal as 
(select player_dismissed as player , count(1) as total_dismissal from ipl_ball_by_ball where dismissal_kind != 'NA' group by player_dismissed)

select runs.player as player from runs left outer join dismissal on runs.player == dismissal.player order by runs.total_runs / coalesce (dismissal.total_dismissal, 1) desc limit 10

#### 8) Write a query to find out who has officiated (as an umpire) the most number of matches in IPL.

In [0]:
%sql
select  umpire1 as umpire , count(umpire1) as umpire_count from ipl_matches group by umpire1 union select umpire2 as umpire , count(umpire2) as umpire_count from ipl_matches group by umpire2 order by umpire_count desc limit 1

### ANALYSIS QUESTION

### SECTION 3

#### 1) Create a database (use any relational DB preferably SQLite) and load data from the dataset (Section 1.1) into db.

#### Section 3.1

###### some imports

In [0]:
import psycopg2
import pandas as pd
import sqlite3

In [0]:
%sql
$sqlite3 create database ipl_database

In [0]:
%sql
use ipl_database

In [0]:
%sql
$sqlite create table ipl_matches
using csv
location '/FileStore/tables/ipl_matches.csv';

In [0]:
%sql
$sqlite create table ipl_ball_by_ball
using csv
location '/FileStore/tables/ipl_ball_by_ball.csv';

In [0]:
%sql
$sqlite create table ipl_venue
using csv
location '/FileStore/tables/ipl_venue.csv';

In [0]:
%sql
 show databases;

databaseName
default
ipl_database
